## 런타임에 체인 내부 구성하기

개발자는 종종 여러 가지 다양한 방식으로 작업을 수행하는 과정이나 실험내용을 최종 사용자(End User)에게 노출하고 싶어 할 때가 있습니다.

이러한 경험을 최대한 쉽게 만들기 위하여 다음의 2가지 메서를 활용할 수 있습니다.

- 첫째, configurable_fields 메서드입니다. 이 메서드를 통해 실행 가능한 객체의 특정 필드를 구성할 수 있습니다.

- 둘째, configurable_alternatives 메서드입니다. 이 메서드를 사용하면 런타임 중에 설정할 수 있는 특정 실행 가능한 객체에 대한 대안을 나열할 수 있습니다.

### configurable_fields

일반적으로 configurable_fields 는 설정 파일, 환경 변수, 또는 사용자 인터페이스를 통해 제공되며, 시스템 관리자나 개발자가 이를 관리합니다.

configurable_fields의 설정할 수 있는 파라미터의 예시로는 데이터베이스 연결 정보, 로깅 설정, 보안 옵션, 성능 튜닝 매개변수 등이 있습니다. 이러한 필드를 적절히 설정함으로써 시스템을 효과적으로 구성하고 관리할 수 있습니다

### 동적 속성 지정

ChatOpenAI 을 사용할 때, 우리는 model_name 와 같은 설정을 조정할 수 있습니다.

model_name 은 GPT 의 버전을 명시할 때 사용하는 속성입니다. 예를 들어, gpt-3.5-turbo, gpt-4 등을 설정하여 모델을 선택할 수 있습니다.

만약, 고정된 model_name 이 아닌 동적으로 모델을 지정하고 싶을 때는 다음과 같이 ConfigurableField 를 활용하여 동적으로 설정할 수 있는 속성 값으로 변환할 수 있습니다.

In [1]:
!pip install -qU langchain langchain-openai

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0).configurable_fields(
    model_name=ConfigurableField(  # model_name 은 원래 ChatOpenAI 의 필드입니다.
        id="gpt_version",  # model_name 의 id 를 설정합니다.
        name="Version of GPT",  # model_name 의 이름을 설정합니다.
        # model_name 의 설명을 설정합니다.
        description="Official model name of GPTs. ex) gpt-3.5-turbo, gpt-4",
    )
)

In [4]:
model.invoke(
    "대한민국의 수도는 어디야?",
    # gpt_version 을 gpt-3.5-turbo 로 설정합니다.
    config={"configurable": {"gpt_version": "gpt-3.5-turbo"}},
)

AIMessage(content='대한민국의 수도는 서울이야.', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 22, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-abc0fd74-74fd-4477-b1f6-0987af901675-0', usage_metadata={'input_tokens': 22, 'output_tokens': 16, 'total_tokens': 38})

In [5]:
model.invoke(
    # gpt_version 을 gpt-4 로 설정합니다.
    "대한민국의 수도는 어디야?",
    config={"configurable": {"gpt_version": "gpt-4"}},
)

AIMessage(content='대한민국의 수도는 서울입니다.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 22, 'total_tokens': 37}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8ff445de-5dbb-4149-9d75-b31986df36ff-0', usage_metadata={'input_tokens': 22, 'output_tokens': 15, 'total_tokens': 37})

In [6]:
model.with_config(configurable={"gpt_version": "gpt-4-turbo-preview"}).invoke(
    "대한민국의 수도는 어디야?"
)

AIMessage(content='대한민국의 수도는 서울입니다.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 22, 'total_tokens': 37}, 'model_name': 'gpt-4-0125-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b4d94b8f-8f9d-4e07-8e6e-3d202d85df6a-0', usage_metadata={'input_tokens': 22, 'output_tokens': 15, 'total_tokens': 37})

In [7]:
# 템플릿에서 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template("{x} 보다 큰 위의 난수를 선택합니다.")
chain = (
    prompt | model
)  # 프롬프트와 모델을 연결하여 체인을 생성합니다. 프롬프트의 출력이 모델의 입력으로 전달됩니다.

In [8]:
chain.invoke({"x": 0})  # 체인을 호출하고 입력 변수 "x"에 0을 전달합니다.

AIMessage(content='5', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 21, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7f3ef108-393e-480d-839d-0b6b4c9dd312-0', usage_metadata={'input_tokens': 21, 'output_tokens': 1, 'total_tokens': 22})

In [9]:
# 체인을 호출할 때 설정을 지정하여 체인을 호출할 수 있습니다.
chain.with_config(configurable={"gpt_version": "gpt-4"}).invoke({"x": 0})

AIMessage(content="죄송합니다만, 제가 난수를 생성하거나 선택하는 기능은 없습니다. 이 기능은 프로그래밍 언어나 소프트웨어를 사용해야 합니다. Python과 같은 프로그래밍 언어에서는 'random' 라이브러리를 사용하여 난수를 생성할 수 있습니다.", response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 21, 'total_tokens': 112}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-74346315-ae9a-4941-b1f7-86f1f49129f6-0', usage_metadata={'input_tokens': 21, 'output_tokens': 91, 'total_tokens': 112})

### HubRunnable: LangChain Hub의 설정 변경

In [10]:
from langchain.runnables.hub import HubRunnable

prompt = HubRunnable("rlm/rag-prompt").configurable_fields(
    # 소유자 저장소 커밋을 설정하는 ConfigurableField
    owner_repo_commit=ConfigurableField(
        # 필드의 ID
        id="hub_commit",
        # 필드의 이름
        name="Hub Commit",
        # 필드에 대한 설명
        description="The Hub commit to pull from",
    )
)

In [11]:
# prompt 객체의 invoke 메서드를 호출하여 "question"과 "context" 매개변수를 전달합니다.
prompt.invoke({"question": "Hello", "context": "World"})

ChatPromptValue(messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: Hello \nContext: World \nAnswer:")])

In [12]:
prompt.with_config(
    # hub_commit 을 teddynote/simple-summary-korean 으로 설정합니다.
    configurable={"hub_commit": "teddynote/simple-summary-korean"}
).invoke({"context": "Hello"})

ChatPromptValue(messages=[HumanMessage(content='주어진 내용을 바탕으로 다음 문장을 요약하세요. 답변은 반드시 한글로 작성하세요\n\nCONTEXT: Hello\n\nSUMMARY:')])

### Configurable Alternatives: Runnable 객체 자체의 대안 설정

구성 가능한 대안들

ChatAnthropic 의 구성 가능한 언어 모델은 다양한 작업과 컨텍스트에 적용할 수 있는 유연성을 제공합니다.

모델의 동작을 제어하기 위해 다음과 같은 구성 옵션을 사용할 수 있습니다.

- model: 사용할 기본 언어 모델을 지정합니다. 예를 들어, "claude-v1", "claude-instant-v1" 등이 있습니다.

- temperature: 0에서 1 사이의 값으로, 샘플링의 무작위성을 제어합니다. 값이 낮을수록 더 결정적이고 반복적인 출력이 생성되며, 값이 높을수록 더 다양하고 창의적인 출력이 생성됩니다.

- max_tokens_to_sample: 생성할 최대 토큰 수를 지정합니다.

- stop_sequences: 지정된 문자열 시퀀스 중 하나가 생성되면 생성을 중지합니다.

- top_k: 샘플링에 고려할 최상위 확률의 토큰 수를 지정합니다.

- top_p: 누적 확률 질량의 상위 부분에서 토큰을 샘플링합니다.

### LLM 객체의 대안(alternatives) 설정 방법

- ChatAnthropic 을 사용하기 위하여 API KEY를 발급받아 설정해야합니다.
- 아래 주석을 해제하고 API KEY를 설정하거나, .env 파일에 설정합니다.

In [13]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatAnthropic
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

llm = ChatAnthropic(temperature=0).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    # 최종 실행 가능한 객체를 구성할 때, 이 id를 사용하여 이 필드를 구성할 수 있습니다.
    ConfigurableField(id="llm"),
    # 기본 키를 설정합니다.
    # 이 키를 지정하면 위에서 초기화된 기본 LLM(ChatAnthropic)이 사용됩니다.
    default_key="anthropic",
    # 'openai'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI()`와 동일합니다.
    openai=ChatOpenAI(),
    # 'gpt4'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI(model="gpt-4")`와 동일합니다.
    gpt4=ChatOpenAI(model="gpt-4"),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)
prompt = PromptTemplate.from_template("{topic} 에 대해 간단히 설명해주세요.")
chain = prompt | llm

c:\Users\anHye\anaconda3\envs\rag\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatAnthropic` was deprecated in LangChain 0.0.28 and will be removed in 0.3. An updated version of the class exists in the langchain-anthropic package and should be used instead. To use it run `pip install -U langchain-anthropic` and import as `from langchain_anthropic import ChatAnthropic`.
  warn_deprecated(


In [19]:
# Anthropic을 기본으로 호출합니다.
chain.invoke({"topic": "뉴진스"})
# 크레딧 잔액 부족으로 실행 불가

In [15]:
# 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"llm": "openai"}).invoke({"topic": "뉴진스"})

AIMessage(content="뉴진스(New Jin's)는 한국의 음식점으로, 전통적인 한식 요리를 제공하는 곳입니다. 주로 한정식이나 다양한 안주 메뉴를 판매하며, 한국의 전통적인 맛을 즐길 수 있는 곳으로 유명합니다. 뉴진스는 한식 요리를 좋아하는 이들에게 인기가 있는 음식점 중 하나입니다.", response_metadata={'token_usage': {'completion_tokens': 124, 'prompt_tokens': 27, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7eb88b14-3ef4-4fe5-84e8-48c7b1c3076e-0', usage_metadata={'input_tokens': 27, 'output_tokens': 124, 'total_tokens': 151})

In [16]:
# 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"llm": "gpt4"}).invoke({"topic": "뉴진스"})

AIMessage(content='"뉴진스"에 대한 구체적인 정보를 제공하기가 어렵습니다. 제공하신 정보가 더 구체적이면 더 정확한 답변을 드릴 수 있을 것 같습니다. 뉴진스가 회사명인지, 제품명인지, 아니면 다른 의미를 가진 용어인지 등에 대한 정보가 필요합니다.', response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 27, 'total_tokens': 136}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3691ce75-a508-467d-b8db-0b66dcc381b8-0', usage_metadata={'input_tokens': 27, 'output_tokens': 109, 'total_tokens': 136})

In [18]:
# 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"llm": "anthropic"}).invoke({"topic": "뉴진스"})
# 크레딧 잔액 부족으로 실행 불가

### 프롬프트의 대안 설정 방법

In [20]:
# 언어 모델을 초기화하고 temperature를 0으로 설정합니다.
llm = ChatOpenAI(temperature=0)

prompt = PromptTemplate.from_template(
    "{country} 의 수도는 어디야?"  # 기본 프롬프트 템플릿
).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    ConfigurableField(id="prompt"),
    # 기본 키를 설정합니다.
    default_key="capital",
    # 'area'이라는 새로운 옵션을 추가합니다.
    area=PromptTemplate.from_template("{country} 의 면적은 얼마야?"),
    # 'population'이라는 새로운 옵션을 추가합니다.
    population=PromptTemplate.from_template("{country} 의 인구는 얼마야?"),
    # 'eng'이라는 새로운 옵션을 추가합니다.
    eng=PromptTemplate.from_template("{input} 을 영어로 번역해주세요."),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)

# 프롬프트와 언어 모델을 연결하여 체인을 생성합니다.
chain = prompt | llm

In [21]:
# config 변경 없이 체인을 호출합니다.
chain.invoke({"country": "대한민국"})

AIMessage(content='대한민국의 수도는 서울이다.', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 23, 'total_tokens': 39}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fdf7b933-ba29-4d8b-a2ad-ed2d24736c4d-0', usage_metadata={'input_tokens': 23, 'output_tokens': 16, 'total_tokens': 39})

In [22]:
# with_config 로 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"prompt": "area"}).invoke({"country": "대한민국"})

AIMessage(content='대한민국의 총 면적은 약 100,363km² 입니다.', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 25, 'total_tokens': 50}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5e1c65a6-a4b7-4043-a2f7-a910d66d2fe9-0', usage_metadata={'input_tokens': 25, 'output_tokens': 25, 'total_tokens': 50})

In [23]:
# with_config 로 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"prompt": "population"}).invoke({"country": "대한민국"})

AIMessage(content='2021년 7월 기준 대한민국의 인구는 약 5천만 명입니다.', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 24, 'total_tokens': 56}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-aaf499dd-53c3-4798-8247-98ba9e1c8667-0', usage_metadata={'input_tokens': 24, 'output_tokens': 32, 'total_tokens': 56})

In [24]:
# with_config 로 체인의 설정을 변경하여 호출합니다.
chain.with_config(configurable={"prompt": "eng"}).invoke({"input": "사과는 맛있어!"})

AIMessage(content='Apples are delicious!', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 28, 'total_tokens': 33}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0f012ed3-5b94-4b54-8fac-1348cc07f897-0', usage_metadata={'input_tokens': 28, 'output_tokens': 5, 'total_tokens': 33})

### 프롬프트 & LLM 모두 변경

In [25]:
llm = ChatAnthropic(temperature=0).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    # 최종 실행 가능한 객체를 구성할 때, 이 id를 사용하여 이 필드를 구성할 수 있습니다.
    ConfigurableField(id="llm"),
    # 기본 키를 설정합니다.
    # 이 키를 지정하면 위에서 초기화된 기본 LLM(ChatAnthropic)이 사용됩니다.
    default_key="anthropic",
    # 'openai'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI()`와 동일합니다.
    openai=ChatOpenAI(),
    # 'gpt4'라는 이름의 새 옵션을 추가하며, 이는 `ChatOpenAI(model="gpt-4")`와 동일합니다.
    gpt4=ChatOpenAI(model="gpt-4"),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)

prompt = PromptTemplate.from_template(
    "{company} 에 대해서 20자 이내로 설명해 줘."  # 기본 프롬프트 템플릿
).configurable_alternatives(
    # 이 필드에 id를 부여합니다.
    ConfigurableField(id="prompt"),
    # 기본 키를 설정합니다.
    default_key="description",
    # 'founder'이라는 새로운 옵션을 추가합니다.
    founder=PromptTemplate.from_template("{company} 의 창립자는 누구인가요?"),
    # 'competitor'이라는 새로운 옵션을 추가합니다.
    competitor=PromptTemplate.from_template("{company} 의 경쟁사는 누구인가요?"),
    # 여기에 더 많은 구성 옵션을 추가할 수 있습니다.
)
chain = prompt | llm

In [26]:
# with_config 로 설정 값을 지정하여 구성할 수 있습니다.
chain.with_config(configurable={"prompt": "founder", "llm": "openai"}).invoke(
    # 사용자가 제공한 회사에 대한 처리를 요청합니다.
    {"company": "애플"}
)

AIMessage(content='애플의 창립자는 스티브 잡스, 스티브 우즈니악, 론 웨인 세 명이 함께 창립했습니다. 하지만 가장 유명한 창립자는 스티브 잡스입니다.', response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 27, 'total_tokens': 110}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-97626945-f846-4478-8583-ce2805e3a1b1-0', usage_metadata={'input_tokens': 27, 'output_tokens': 83, 'total_tokens': 110})

In [28]:
# 하나만 구성하려는 경우
chain.with_config(configurable={"llm": "anthropic"}).invoke({"company": "애플"})
# 크레딧 부족으로 실행 불가

In [30]:
# 하나만 구성하려는 경우
chain.with_config(configurable={"prompt": "competitor"}).invoke({"company": "애플"})
# 크레딧 부족으로 실행 불가

In [32]:
# 하나만 구성하려는 경우
chain.invoke({"company": "애플"})
# 크레딧 부족으로 실행 불가

### 설정 저장

In [33]:
# with_config 로 설정을 변경하여 생성한 체인을 별도의 변수에 저장합니다.
gpt4_competitor_chain = chain.with_config(
    configurable={"llm": "gpt4", "prompt": "competitor"}
)

In [34]:
# 체인을 호출합니다.
gpt4_competitor_chain.invoke({"company": "애플"})

AIMessage(content='애플의 주요 경쟁사로는 삼성전자, 구글, 마이크로소프트, 화웨이, LG전자 등이 있습니다. 이들 회사는 스마트폰, 컴퓨터, 태블릿, 웨어러블 기기, 소프트웨어 서비스 등에서 애플과 경쟁하고 있습니다.', response_metadata={'token_usage': {'completion_tokens': 111, 'prompt_tokens': 27, 'total_tokens': 138}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cbbd6c29-1587-4004-8ab6-e9847445208f-0', usage_metadata={'input_tokens': 27, 'output_tokens': 111, 'total_tokens': 138})